In [2]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import numpy as np

ITEM_COUNTS = 100

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'Display-Api-Key': 'VWmkUPgs6g2fviPZ5JQFQ3pERP4tIXv/J2jppLqSRBk='
}

In [4]:
middle_category_nums = ['10101201', '10101202', '10101203', '10101204', '10101205',
                        '10101206', '10101207', '10101208', '10101209', '10101210',
                        '10101211', '10101212']

In [5]:
# def get_item_cds_fp(middle_category_num, gender):
#     url = 'https://api-display.wconcept.co.kr/display/api/v2/best/products'

#     if gender == 'men':
#         genderType = 'men'
#     else:
#         genderType = 'women'

#     data = {
#     "custNo": "",
#     "dateType": "daily",
#     "domain": 'WOMEN',
#     "genderType": genderType,
#     "depth1Code": "10101",
#     "depth2Code": middle_category_num,
#     "pageNo": 1,
#     "pageSize": ITEM_COUNTS
#     }

#     response = requests.post(url, headers=headers, json=data)

#     soup = BeautifulSoup(response.text, 'lxml')
#     info = soup.string
#     info = json.loads(info)
#     info = info['data']['content']

#     item_cds_fp = []
    
#     ranking = 1

#     for item in info:
#         item_cds_fp.append([item['itemCd'], item['finalPrice'], ranking])
#         ranking += 1

#     return item_cds_fp

In [6]:
def get_item_cds_fp(middle_category_num, gender):
    url = 'https://api-display.wconcept.co.kr/display/api/v2/best/products'

    if gender == 'men':
        genderType = 'men'
    else:
        genderType = 'women'

    data = {
    "custNo": "",
    "dateType": "daily",
    "domain": 'WOMEN',
    "genderType": genderType,
    "depth1Code": "10101",
    "depth2Code": middle_category_num,
    "pageNo": 1,
    "pageSize": ITEM_COUNTS
    }

    response = requests.post(url, headers=headers, json=data)

    soup = BeautifulSoup(response.text, 'lxml')
    info = soup.string
    info = json.loads(info)
    info = info['data']['content']

    item_cds_fp = []

    for item in info:
        item_cds_fp.append([item['itemCd'], item['finalPrice']])
        
    item_cds_fp = np.array(item_cds_fp)
    rankings = np.arange(1, len(item_cds_fp) + 1)
    total_items_count = np.full(len(item_cds_fp), len(item_cds_fp))

    item_cds_fp = np.column_stack((item_cds_fp, rankings, total_items_count))

    return item_cds_fp.tolist()

In [7]:
def get_item_data(info, item_fp):
    # 브랜드
    brandNameKr = info['brandNameKr']

    # 품번
    itemCd = info['itemCd']

    # 좋아요
    heartCnt = info['heartCnt']

    # 품절 여부
    statusName = info['statusName']

    # 정상가
    customerPrice = info['customerPrice']

    # 쿠폰 적용가 
    finalPrice = item_fp

    # 색상

    # 사이즈

    # 이미지
    url = f'https://www.wconcept.co.kr/Product/{itemCd}?rccode=pc_topseller'
    response = requests.get(url, headers)
    soup = BeautifulSoup(response.text, 'lxml')
    imageUrls = []
    try:
        images = soup.select('ul#gallery > li > a > img') 
        for image in images:
            imageUrls.append(image['src'])

    except:
        pass

    # 상품명
    itemName = info['itemName']

    data = {
        'brandNameKr': brandNameKr,
        'itemCd': itemCd,
        'heartCnt': heartCnt,
        'statusName': statusName,
        'customerPrice': customerPrice,
        'finalPrice': finalPrice,
        'itemName': itemName,
        'imageUrls': imageUrls
    }

    return data



In [8]:
def get_item_info(item_cd, item_fp):
    
    url = f'https://www.wconcept.co.kr/Ajax/GetProductsInfo'
    data = {'itemcds': item_cd}
    response = requests.post(url, headers=headers, data=data)

    soup = BeautifulSoup(response.text, 'lxml')
    info = json.loads(soup.string)[0]
    
    info = get_item_data(info, item_fp)

    return info



In [9]:
def get_rank_score(ranking, item_count):
    rank_score = 1 - ((ranking - 1) / (item_count - 1))

    return rank_score

In [12]:
def main():
    item_cds_list = []
    for gender in ['men', 'women']:
        for middle_category_num in middle_category_nums:
            item_cds_fp = get_item_cds_fp(middle_category_num, gender)
            item_cds_list += item_cds_fp
    
    item_info = []
    for item_cd, item_fp, ranking, middle_item_count in item_cds_list:
        info = get_item_info(item_cd, item_fp)
        info['rank_score'] = get_rank_score(int(ranking), int(middle_item_count))
        item_info.append(info)
        print(info)
    df = pd.DataFrame(item_info)
    df.to_csv('wconcept.csv', index=False)
    

In [13]:
if __name__ == '__main__':
    main()

{'brandNameKr': '코닥 어패럴', 'itemCd': '305685943', 'heartCnt': 19.0, 'statusName': '판매중', 'customerPrice': 189000.0, 'finalPrice': '170100', 'itemName': '파인더 후드 아노락 자켓 IVORY', 'imageUrls': ['//product-image.wconcept.co.kr/productimg/image/img0/43/305685943_NG18313.jpg', '//product-image.wconcept.co.kr/productimg/image/img0/43/305685943_add1_IH16136.jpg?thumbnail=60x80'], 'rank_score': 1.0}
{'brandNameKr': '필시크', 'itemCd': '305814488', 'heartCnt': 0.0, 'statusName': '판매중', 'customerPrice': 278000.0, 'finalPrice': '225180', 'itemName': 'MINIMAL SHORT JACKET (BLACK)', 'imageUrls': ['//product-image.wconcept.co.kr/productimg/image/img0/88/305814488_UO61336.jpg', '//product-image.wconcept.co.kr/productimg/image/img0/88/305814488_add1_JL79048.jpg?thumbnail=60x80', '//product-image.wconcept.co.kr/productimg/image/img0/88/305814488_add2_QP97836.jpg?thumbnail=60x80', '//product-image.wconcept.co.kr/productimg/image/img0/88/305814488_add3_OO10104.jpg?thumbnail=60x80'], 'rank_score': 0.982456140350

In [ ]:
df = pd.read_csv('wconcept.csv')

In [ ]:
df

,brandNameKr,itemCd,heartCnt,statusName,customerPrice,finalPrice,itemName,imageUrls
0,웬즈데이오아시스,302746077,1607.0,판매중,57350.0,40145,overfit two-way hoodie zipup(4col),['//product-image.wconcept.co.kr/productimg/im...
1,팔육디,304069749,17.0,판매중,619000.0,341400,STITCH LEATHER JACKET (DARK BROWN),['//product-image.wconcept.co.kr/productimg/im...
2,레씨토,301320113,2.0,판매중,269000.0,269000,SUMMER WOOL DADDY FIT 2-BUTTON BLAZER_[GRAYISH...,['//product-image.wconcept.co.kr/productimg/im...
3,코닥 어패럴,305686017,24.0,판매중,199000.0,173130,씨너리 컬러블록 아노락 자켓 IVORY,['//product-image.wconcept.co.kr/productimg/im...
4,빅유니온,302328438,214.0,판매중,98000.0,47040,89LINE ZIP UP HOODIE / OATMEAL,['//product-image.wconcept.co.kr/productimg/im...
...,...,...,...,...,...,...,...,...
1583,허그데이,301885944,36.0,판매중,12500.0,11900,어나더 소프트 코튼 와이드 나시,['//product-image.wconcept.co.kr/productimg/im...
1584,캘빈클라인 언더웨어,300919590,600.0,판매중,35000.0,35000,여성 모던코튼 비키니 팬티_F3787AD_001,['//product-image.wconcept.co.kr/productimg/im...
1585,휘게 언더웨어,302931877,64.0,판매중,45000.0,40500,소프트 노와이어 브라 팬티 세트 5 Colors,['//product-image.wconcept.co.kr/productimg/im...
1586,세컨스킨,305740010,18.0,판매중,56700.0,39690,데일리 베이직 캐미브라 3종,['//product-image.wconcept.co.kr/productimg/im...
